## Get Started

In [22]:
import pandas as pd
from floweaver import *

In [2]:
df = pd.read_csv("WorldCruiseData.csv")

In [3]:
df

,ROUTE,EMBARK_PORT,DISEMBARK_PORT,CABIN_TYPE,DURATION,CRUISE_REGION,BOOKING_WEEK_PRIOR,CANCEL_WEEK_PRIOR,EMBARK_REGION,DEBARK_REGION,PRICE_PAID,NO_OF_PEOPLE_ON_BOOKING
0,SOU-AKL,SOU,AKL,O,40,West Bound,33,33.0,UK,New Zealand,2811.127882,2
1,SOU-AKL,SOU,AKL,O,40,West Bound,45,NaN,UK,New Zealand,5171.396346,2
2,SOU-AKL,SOU,AKL,B,40,West Bound,42,NaN,UK,New Zealand,6343.673826,2
3,SOU-BNE,SOU,BNE,B,47,West Bound,36,35.0,UK,Australia - Queensland,9784.513217,2
4,SOU-SFO,SOU,SFO,B,24,Panama Canal,57,NaN,UK,USA - West Coast,4182.892388,2
...,...,...,...,...,...,...,...,...,...,...,...,...
3918,HKG-SOU,HKG,SOU,B,36,Southeast Asia,42,41.0,China - Southern,UK,5084.221402,2
3919,DXB-SOU,DXB,SOU,S,19,Middle East,47,NaN,UAE,UK,5636.195082,2
3920,DXB-SOU,DXB,SOU,B,19,Middle East,24,NaN,UAE,UK,1596.550657,2
3921,DXB-SOU,DXB,SOU,B,19,Middle East,12,12.0,UAE,UK,2160.742284,2


In [4]:
flows = (
    df.groupby(["EMBARK_PORT", "DISEMBARK_PORT", "CRUISE_REGION"])
    .agg({"PRICE_PAID": "mean"})
    .dropna()
    .reset_index()
    .rename(
        columns={
            "EMBARK_PORT": "source",
            "DISEMBARK_PORT": "target",
            "CRUISE_REGION": "type",
            "PRICE_PAID": "value",
        }
    )
)

In [5]:
nodes = {
    "embark_port": ProcessGroup(flows["source"].unique().tolist()),
    "disembark_port": ProcessGroup(flows["target"].unique().tolist()),
}

In [6]:
ordering = [["embark_port"], ["disembark_port"]]

In [7]:
bundles = [Bundle("embark_port", "disembark_port")]

In [8]:
sdd = SankeyDefinition(nodes, bundles, ordering)
weave(sdd, flows).to_widget().auto_save_png("simple.png")

SankeyWidget(layout=Layout(height='500', width='700'), links=[{'source': 'embark_port^*', 'target': 'disembark…

In [9]:
# Create partitions for each type of port
ports = ["AKL", "BNE", "DXB", "HKG", "SFO", "SIN", "SOU", "SYD"]

embark_port = Partition.Simple("process", ports)
disembark_port = Partition.Simple("process", ports)

nodes["embark_port"].partition = embark_port
nodes["disembark_port"].partition = disembark_port

# Create Sankey diagram
weave(sdd, flows).to_widget().auto_save_png("area.png")

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^AKL', 'embar…

## Combine Different Categories

In [10]:
continents = [
    ("Oceania", ["AKL"]),
    ("Australia", ["BNE", "SYD"]),
    ("North America", ["SFO"]),
    ("Europe", ["SOU"]),
    ("Asia", ["HKG", "SIN", "DXB"]),
]

In [11]:
continent_groups = Partition.Simple("process", continents)

In [12]:
nodes["embark_port"].partition = continent_groups
nodes["disembark_port"].partition = continent_groups

In [13]:
weave(sdd, flows).to_widget().auto_save_png("continent.png")

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

## Color Intensity

In [14]:
weave(sdd, flows, link_color=QuantitativeScale("value")).to_widget().auto_save_png(
    "intensity.png"
)

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

In [15]:
weave(
    sdd, flows, link_color=QuantitativeScale("value", palette="Blues_9")
).to_widget().auto_save_png("intensity.png")

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

## Distinguish Different Flow Types

In [16]:
cruise_region = Partition.Simple("type", flows.type.unique())

sdd = SankeyDefinition(
    nodes,
    bundles,
    ordering,
    flow_partition=cruise_region,
)
weave(sdd, flows).to_widget().auto_save_png("cruise_region.png")

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…

## Group Lines of the Same Type

In [17]:
# 1. Define a new waypoint node
nodes["waypoint"] = Waypoint(cruise_region)

# 2. Update the ordering to show where the waypoint goes: in the middle
ordering = [
    ["embark_port"],
    ["waypoint"],
    ["disembark_port"],
]

# 3. Update the bundle definition to send the flows via the waypoint
bundles = [
    Bundle("embark_port", "disembark_port", waypoints=["waypoint"]),
]

# Update the SDD with the new nodes, ordering & bundles.
sdd = SankeyDefinition(nodes, bundles, ordering)

weave(sdd, flows, link_color=QuantitativeScale("value", palette='Blues_9')).to_widget().auto_save_png(
    "waypoint.png"
)

SankeyWidget(groups=[{'id': 'embark_port', 'type': 'process', 'title': '', 'nodes': ['embark_port^Oceania', 'e…